In [ ]:
!pip3 install onnx
!pip3 install tensorflow
!pip3 install nilearn
!pip3 install scikit-learn
!pip3 install --upgrade scikit-learn imbalanced-learn
!pip3 install pytest
!pip3 install nipype
!pip3 install --upgrade tensorflow keras


In [ ]:
!pip3 install --upgrade tensorflow

In [1]:
import os
os.environ['TF_KERAS'] = '1'

import onnx
import numpy as np
import re
from collections import Counter
import tensorflow as tf
import glob
from nilearn.input_data import NiftiMasker
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, LeaveOneGroupOut
from keras.layers import TimeDistributed, LSTM
from nilearn.masking import apply_mask, unmask, compute_epi_mask
from nilearn.image import resample_to_img, index_img
from nibabel.testing import data_path
import nibabel as nib
import nilearn
import argparse
import keras
import keras.backend as K
import keras.layers
import keras.models
import keras.utils
from keras.optimizers import Adam
import nipype.interfaces.io as nio
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.io as sio
from nilearn import image, plotting
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# For TensorFlow 1.x compatibility
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Use to_categorical from tensorflow.keras.utils instead of np_utils
from tensorflow.keras.utils import to_categorical


Instructions for updating:
non-resource variables are not supported in the long term


# KKI DATASET


In [2]:
# Getting labels
phenotypic = pd.read_csv('/Users/buketgencaydin/Downloads/data/KKI_phenotypic.csv')
labels = phenotypic['DX']
labels = pd.DataFrame(labels)
ID = phenotypic['ScanDir ID']
ID = pd.DataFrame(ID)
result = pd.concat([ID, labels], axis=1, join='inner')


In [3]:
# Getting fMRI image path of one KKI site
nifti_files = glob.glob('/Users/buketgencaydin/Downloads/data/fmri_KKI/*')

In [4]:
# Separating IDs from the list of paths
s_nifti = []
for i in nifti_files:
    s_nifti.append(re.split('(\d+)', i))

In [5]:
#separting ID from the list of path 
s1_nifti = []
for i in range(len(s_nifti)):
    s1_nifti.append(s_nifti[i][1])

In [6]:
import numpy as np
import nibabel as nb

# Load data
nii = nb.load(nifti_files[0])
img = nii.get_fdata()

# Get voxel dimensions
voxel_dims = (nii.header["pixdim"])[1:4]
print("Voxel dimensions:")
print("  x = {} mm".format(voxel_dims[0]))
print("  y = {} mm".format(voxel_dims[1]))
print("  z = {} mm".format(voxel_dims[2]))

# Compute volume
nonzero_voxel_count = np.count_nonzero(img)
voxel_volume = np.prod(voxel_dims)
nonzero_voxel_volume = nonzero_voxel_count * voxel_volume

print("Number of non-zero voxels = {}".format(nonzero_voxel_count))
print("Volume of non-zero voxels = {} mm^3".format(nonzero_voxel_volume))

Voxel dimensions:
  x = 3.0 mm
  y = 3.0 mm
  z = 3.0 mm
Number of non-zero voxels = 18927572
Volume of non-zero voxels = 511044444.0 mm^3


In [7]:
from nilearn.input_data import NiftiMasker
from nilearn.image import index_img


epi_img = index_img(nifti_files[0], slice(0, 100))
masker = NiftiMasker(mask_strategy='epi')
s1 = masker.fit(epi_img)
type(s1)

nilearn.maskers.nifti_masker.NiftiMasker

In [8]:
fmri_masked.shape

NameError: name 'fmri_masked' is not defined

In [ ]:
import nibabel as nib
from nilearn import image

# Load the first NIfTI file
Img_1 = nib.load(nifti_files[0])

# Index the image
Niak_1 = image.index_img(Img_1, 1)

# Print the shape of the indexed image
print(Niak_1.shape)

# Retrieve the header information
Niak_info = Niak_1.header
print(Niak_info)

In [9]:
## sorting labels according to file

Id = result['ScanDir ID'].values.tolist()
label = result['DX'].values.tolist()
#df2 = s1_nifti[0].values.tolist() 

df_sort = []
for i in range(len(s1_nifti)):
    df_sort.append(int(s1_nifti[i]))

In [10]:
#sorting the labels according to file

new_list = []
for i in range(len(df_sort)):
    if df_sort[i] in Id:
        s = Id.index(df_sort[i])
        #print(df_sort[i])
        new_list.append(label[s])
        #print(label[s])

In [11]:
Img = nib.load('/Users/buketgencaydin/Downloads/data/KKI_data/' + s1_nifti[2] +'.nii.gz')
x = Img.get_fdata()
x.shape

(53, 64, 46)

In [ ]:
#Preprocessing Visualization of one instance without ADHD
data = arr 
mean_func = image.mean_img(Niak_1)

#smoothing  with a varying amount of smoothing, from none to 14mm by increment of 2mm
for smoothing in range(0, 3, 1):
    smoothed_img = image.smooth_img(mean_func, smoothing)
    plotting.plot_epi(smoothed_img,title="Smoothing %imm" % smoothing)

In [ ]:
#Performing smoothing and cleaning of image on whole dataset and saving it to another folder
for i in range(len(nifti_files)):
    smoothed_img = image.smooth_img(nifti_files[i], 2)
    s = image.clean_img(smoothed_img, runs=None, detrend=True ,standardize= True)
    s2 = image.index_img(s,10)
    nib.save(s2 , '/Users/buketgencaydin/Downloads/data/KKI_data/' + s1_nifti[i] +'.nii.gz')

In [29]:
#performing 2mm smoothing on whole data
#reshaping the dimensions of the data before passing to  cnn
Dataset = []
for i in range(83):
    Img = nib.load('/Users/buketgencaydin/Downloads/data/KKI_data/' + s1_nifti[i] +'.nii.gz')
    x = Img.get_fdata()
    #result = x[:, :, :,:1]
    Dataset.append(x)

In [ ]:
#reshaping the dimensions of the data before passing to  cnn
Dataset = []
for i in range(83):
    Img = nib.load('/Users/buketgencaydin/Downloads/data/KKI_data/' + s1_nifti[i] +'.nii.gz')
    x = Img.get_fdata()
    #result = x[:, :, :,:1]
    Dataset.append(x)

In [19]:
X = np.array(Dataset)
Y = np.array(new_list)

In [30]:
#Binary classification
for i in range(len(new_list)):
    if new_list[i] > 1:
        new_list[i] = 1
    else:
        continue 
        
#X = np.array([Dataset], dtype = object)
#Y = np.array(new_list)

In [31]:
Counter(Y)

Counter({0: 61, 1: 22})

In [32]:
X.shape

(83, 53, 64, 46)

In [33]:
#combining both dataset
#x_combined = np.concatenate((X, X_ohsu))
#y_combined = np.concatenate((Y,Y_ohsu))

x_combined = X
y_combined = Y

In [34]:
#using counter to check if data is balanced or not
Counter(y_combined)

Counter({0: 61, 1: 22})

In [37]:
#reshaping the train data value
reshaped_data = []
for i in range(len(x_combined)):
    reshaped_data.append(x_combined[i].reshape(1,53,64,46))
X_data = np.array(reshaped_data)

In [38]:
#converting to categorical
Y_new = keras.utils.to_categorical(y_combined, 4)


In [47]:
#splitting the dataset
x_train, x_test, y_train, y_test = train_test_split(X_data,Y_new,shuffle = True, random_state = 32, test_size=0.15)



In [44]:
from keras.models import Sequential
from keras.layers import TimeDistributed, Conv2D, Activation, MaxPool2D, Dropout, Flatten, LSTM, Dense
from keras.optimizers import Adam

# Define the model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, kernel_size=3, kernel_initializer='uniform', padding='same'),
                          input_shape=(None, 53, 64, 46)))  # Ensure this input shape matches your data

model.add(TimeDistributed(Activation("relu")))
model.add(TimeDistributed(MaxPool2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(32, kernel_size=3, kernel_initializer='uniform', padding='same', activation='relu')))
model.add(TimeDistributed(MaxPool2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(32, kernel_size=3, kernel_initializer='uniform', padding='same', activation='relu')))
model.add(TimeDistributed(MaxPool2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(32, kernel_size=3, kernel_initializer='uniform', padding='same', activation='relu')))
model.add(TimeDistributed(MaxPool2D(pool_size=2)))

model.add(TimeDistributed(Conv2D(32, kernel_size=3, kernel_initializer='uniform', padding='same', activation='relu')))

model.add(TimeDistributed(Conv2D(32, kernel_size=3, kernel_initializer='uniform', padding='same', activation='relu')))

model.add(TimeDistributed(Dropout(0.2)))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))

model.add(Dense(250, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(250, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(4))
model.add(Activation('softmax'))

# Initialize the Adam optimizer
adam = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.1)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Print the model summary
#print(model.summary())


In [45]:
model.fit(x_train, y_train,epochs= 200, verbose=1)

Epoch 1/200


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
#cross validation
Kfold = KFold(n_splits=7)
Kfold.get_n_splits(x_train, y_train)  
foldNum=0                                  #initializing fold = 0
for train_index, val_index in Kfold.split(x_train, y_train):
    foldNum+=1
    print("fold",foldNum)
    X_train, X_val = x_train[train_index], x_train[val_index]
    Y_train, Y_val = y_train[train_index], y_train[val_index]
    print(X_train.shape)
    #training the dataset
    history1 = model.fit(X_train, Y_train, validation_data = (X_val,Y_val), epochs=20,batch_size=32, verbose=1)

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
#model.predict(x_test)

In [ ]:
model.save('FMRI_model.h5') 

In [ ]:
model.save_weights('FMRI_weights.h5')